In [1]:
import pydicom
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops, marching_cubes, mesh_surface_area
from skimage.segmentation import clear_border
from skimage.morphology import remove_small_objects, binary_opening, binary_closing, disk
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial import Delaunay


In [ ]:
# pip install pydicom
# pip install numpy

In [ ]:
def load_dicom_series(folder_path):
    """
    Load a DICOM image series from a folder and return the 3D volume and voxel spacing.

    Parameters:
    - folder_path (str): Path to the folder containing DICOM files.

    Returns:
    - volume (numpy array): 3D NumPy array of image data.
    - spacing (tuple): (slice thickness, pixel spacing in x, pixel spacing in y).
    """
    dicom_files = []

    # Load DICOM files from folder
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.dcm'):
            filepath = os.path.join(folder_path, filename)
            try:
                dicom_files.append(pydicom.dcmread(filepath))
            except Exception as e:
                print(f"⚠️ Warning: Could not read {filename} - {e}")

    if not dicom_files:
        raise ValueError("❌ No valid DICOM files found in the folder.")

    # Try sorting by ImagePositionPatient if available
    try:
        dicom_files.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    except AttributeError:
        print("⚠️ ImagePositionPatient not found. Using default order.")

    # Stack images into a 3D NumPy array
    volume = np.stack([file.pixel_array for file in dicom_files])

    # Extract voxel spacing
    try:
        slice_thickness = float(dicom_files[0].SliceThickness)
    except AttributeError:
        slice_thickness = 1.0  # Default value if missing
        print("⚠️ SliceThickness not found. Using default value of 1.0.")

    try:
        pixel_spacing = dicom_files[0].PixelSpacing
        spacing = (slice_thickness, float(pixel_spacing[0]), float(pixel_spacing[1]))
    except AttributeError:
        spacing = (slice_thickness, 1.0, 1.0)  # Default pixel spacing
        print("⚠️ PixelSpacing not found. Using default (1.0, 1.0).")

    print(f"Loaded DICOM series: {len(dicom_files)} slices")
    print(f"Volume shape: {volume.shape}")
    print(f"Voxel spacing: {spacing}")

    return volume, spacing

# Example usage for patient SCD0000101
# folder_path = r"C:\Users\Kaiwen Liu\OneDrive - University of Toronto\Desktop\github_repo\heart_cardiac_mri_image_processing\data\SCD_IMAGES_01\SCD0000101\CINESAX_300" # Change depending on where you patient files are
folder_path = r"C:\Users\Kaiwen Liu\OneDrive - University of Toronto\Desktop\github_repo\heart_cardiac_mri_image_processing\data\SCD_IMAGES_05\SCD0004501\CINESAX_1100"

volume, spacing = load_dicom_series(folder_path)



In [ ]:
def normalize_volume(volume):
    # Clip out extreme values (optional, helps with outliers)
    volume = np.clip(volume, np.percentile(volume, 1), np.percentile(volume, 99))

    # Min-max normalization to [0, 1]
    volume = volume.astype(np.float32)
    volume -= volume.min()
    volume /= volume.max()

    return volume


In [ ]:
def apply_center_weighting(volume, alpha=0.5):
    """
    Enhance the image intensity towards the center using a Gaussian weight map.

    Parameters:
    - volume: 3D MRI volume.
    - alpha: Weighting factor (0.0 - 1.0).
    """
    z, y, x = volume.shape

    # Create a Gaussian weight map centered in the middle
    yy, xx = np.meshgrid(np.linspace(-1, 1, x), np.linspace(-1, 1, y))
    distance = np.sqrt(xx**2 + yy**2)
    weight_map = np.exp(-4 * distance**2)  # Gaussian-like falloff

    weighted_volume = np.zeros_like(volume)

    # Apply the weight map to each slice
    for i in range(z):
        weighted_volume[i] = volume[i] * (1 + alpha * weight_map)

    return weighted_volume

In [ ]:
def crop_center(volume, crop_size):
    """
    Crop the center region of a 3D volume.

    Parameters:
    - volume: 3D NumPy array (z, y, x).
    - crop_size: Tuple (crop_height, crop_width).

    Returns:
    - Cropped volume.
    """
    z, y, x = volume.shape
    crop_height, crop_width = crop_size

    # Calculate center coordinates
    center_y, center_x = y // 2, x // 2

    # Define cropping boundaries
    y_min = max(center_y - crop_height // 2, 0)
    y_max = min(center_y + crop_height // 2, y)
    x_min = max(center_x - crop_width // 2, 0)
    x_max = min(center_x + crop_width // 2, x)

    # Crop the volume
    cropped_volume = volume[:, y_min:y_max, x_min:x_max]

    return cropped_volume

In [ ]:
def remove_border_components(binary_slice):
    """
    Remove connected components touching the borders of the image.

    Parameters:
    - binary_slice (numpy array): A 2D binary image.

    Returns:
    - cleaned_slice (numpy array): Binary image with border-touching components removed.
    """
    # Label connected components in the binary mask
    labeled_slice = label(binary_slice)

    # Remove components touching the image borders
    cleaned_slice = clear_border(labeled_slice)

    # Convert back to binary
    return cleaned_slice > 0

def segment_heart_otsu(volume):
    """
    Segment the heart using Otsu's thresholding and remove border-touching components.

    Steps:
    1. Apply Otsu's thresholding to segment potential heart regions.
    2. Remove any connected components touching the image borders.
    3. Return the cleaned segmented volume.

    Parameters:
    - volume (numpy array): 3D NumPy array containing the image stack.

    Returns:
    - segmented_volume (numpy array): Binary volume with only the heart region.
    """

    segmented_volume = np.zeros_like(volume, dtype=bool)  # Initialize output

    for i in range(volume.shape[0]):  # Iterate over slices
        slice_img = volume[i]

        # Step 1: Apply Otsu’s thresholding
        thresh = threshold_otsu(slice_img)
        binary_slice = slice_img > thresh  # Convert to binary mask

        # Step 2: Remove border-touching components
        cleaned_slice = remove_border_components(binary_slice)

        # Store in output volume
        segmented_volume[i] = cleaned_slice

    return segmented_volume.astype(np.uint8)  # Convert to uint8 for visualization

In [ ]:
def filter_circular_regions(segmented_volume, circularity_thresh=0.6, min_size=300):
    """
    Keep only circular regions in a segmented 3D volume.

    Parameters:
    - segmented_volume: 3D binary NumPy array (segmentation mask).
    - circularity_thresh: Circularity threshold (higher = more circular).
    - min_size: Minimum region size to keep (removes small noise).

    Returns:
    - Filtered 3D binary mask with mostly circular regions.
    """
    filtered_volume = np.zeros_like(segmented_volume)

    for z in range(segmented_volume.shape[0]):
        slice_mask = segmented_volume[z]

        # Label connected components
        labeled_mask = label(slice_mask)

        for region in regionprops(labeled_mask):
            # Calculate circularity: (4 * pi * area) / perimeter^2
            if region.perimeter > 0:  # Avoid division by zero
                circularity = (4 * np.pi * region.area) / (region.perimeter ** 2)

                # Keep region if circular enough and above min size
                if circularity >= circularity_thresh and region.area >= min_size:
                    filtered_volume[z][labeled_mask == region.label] = 1

    return filtered_volume

In [ ]:
import numpy as np
import cv2
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import watershed
from skimage.filters import sobel
from skimage.measure import label

def improved_watershed(segmented_volume):
    """
    Applies an enhanced Watershed segmentation across all slices in a 3D volume
    to separate heart chambers more effectively.

    Parameters:
    - segmented_volume (ndarray): Binary 3D array from Otsu thresholding.

    Returns:
    - watershed_labels (ndarray): 3D array with labeled regions after Watershed segmentation.
    """

    # Initialize output volume
    watershed_labels = np.zeros_like(segmented_volume, dtype=np.int32)

    for i in range(segmented_volume.shape[0]):  # Process each slice
        binary_slice = segmented_volume[i]

        if np.sum(binary_slice) == 0:
            continue  # Skip empty slices

        # Compute edges using the Sobel filter (gradient magnitude)
        edges = sobel(binary_slice)

        # Distance transform (higher values for inner areas)
        distance_map = distance_transform_edt(binary_slice)

        # Threshold distance map to get seeds (use 40% of max)
        seed_threshold = 0.4 * np.max(distance_map)
        seeds = distance_map > seed_threshold

        # Label connected components in the seeds
        markers = label(seeds)

        # Apply Watershed with edges as the barrier
        ws_result = watershed(edges, markers, mask=binary_slice)

        # Store result
        watershed_labels[i] = ws_result

    return watershed_labels

In [ ]:


def LV_isolation(watershed_labels, max_size_threshold=2000):
    """
    Selects the Left Ventricle (LV) by first identifying the most prominent LV region
    in the center slices and then extending this bias to the slices above and below.
    A size filter is applied to remove large non-LV structures.

    Parameters:
    - watershed_labels (ndarray): 3D labeled array from Watershed segmentation.
    - original_volume (ndarray): The original grayscale MRI volume.
    - max_size_threshold (int): Maximum allowable region size; larger regions are removed.

    Returns:
    - lv_volume (ndarray): 3D binary array containing only the LV segmentation.
    """

    lv_volume = np.zeros_like(watershed_labels, dtype=np.uint8)
    num_slices = watershed_labels.shape[0]
    center_slice_idx = num_slices // 2  # Middle slice index

    # Step 1: Identify LV in the center slices
    center_range = range(center_slice_idx - 2, center_slice_idx + 3)  # Use ±2 slices around center
    best_lv_centroid = None

    for i in center_range:
        slice_labels = watershed_labels[i]
        labeled_regions = label(slice_labels)

        if np.max(labeled_regions) == 0:
            continue  # Skip empty slices

        # Get properties of the segmented regions
        regions = [r for r in regionprops(labeled_regions) if r.area <= max_size_threshold]

        if not regions:
            continue

        # Find the largest central region (assuming LV is well-segmented here)
        best_region = max(regions, key=lambda r: r.area)
        best_lv_centroid = best_region.centroid  # Store centroid for biasing other slices

        # Assign the LV label in the center slices
        lv_volume[i] = (labeled_regions == best_region.label)

    if best_lv_centroid is None:
        raise ValueError("Could not identify a central LV region. Check the segmentation.")

    # Step 2: Extend LV detection above and below using centroid bias
    best_x, best_y = best_lv_centroid  # LV centroid coordinates in the middle slices

    for i in range(num_slices):
        if i in center_range:
            continue  # Skip already processed center slices

        slice_labels = watershed_labels[i]
        labeled_regions = label(slice_labels)

        if np.max(labeled_regions) == 0:
            continue  # Skip empty slices

        regions = [r for r in regionprops(labeled_regions) if r.area <= max_size_threshold]
        best_region = None
        min_distance = float("inf")

        # Find the closest region to the LV centroid
        for region in regions:
            centroid_x, centroid_y = region.centroid
            distance = np.sqrt((centroid_x - best_x) ** 2 + (centroid_y - best_y) ** 2)

            if distance < min_distance:
                min_distance = distance
                best_region = region

        if best_region:
            lv_volume[i] = (labeled_regions == best_region.label)


    return lv_volume


In [ ]:
def clean_segmentation(volume, radius=1):
    struct_elem = disk(radius)  # 2D circular element

    cleaned_volume = np.zeros_like(volume)
    for i in range(volume.shape[0]):
        cleaned_slice = binary_closing(binary_opening(volume[i], struct_elem), struct_elem)
        cleaned_volume[i] = cleaned_slice

    return cleaned_volume

In [ ]:


def plot_lv_metrics(lv_volumes, lv_surface_areas, num_frames=20):
    """
    Plots LV Volume (mL) and LV Surface Area (cm²) over the cardiac cycle.

    Parameters:
    - lv_volumes (list): List of LV volumes over time.
    - lv_surface_areas (list): List of LV surface areas over time.
    - num_frames (int): Number of frames in the cardiac cycle (default: 20).
    """
    time_points = np.linspace(0, 100, num_frames)  # Normalize time (0-100% cardiac cycle)

    plt.figure(figsize=(10, 5))

    # Plot LV Volume
    plt.subplot(1, 2, 1)
    plt.plot(time_points, lv_volumes, marker="o", linestyle="-", color="b", label="LV Volume")
    plt.xlabel("Cardiac Cycle (%)")
    plt.ylabel("LV Volume (mL)")
    plt.title("LV Volume over Cardiac Cycle")
    plt.legend()
    plt.grid(True)

    # Plot LV Surface Area
    plt.subplot(1, 2, 2)
    plt.plot(time_points, lv_surface_areas, marker="s", linestyle="-", color="r", label="LV Surface Area")
    plt.xlabel("Cardiac Cycle (%)")
    plt.ylabel("LV Surface Area (cm²)")
    plt.title("LV Surface Area over Cardiac Cycle")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

def extract_surface_marching_cubes(original_volume, segmented_volume, spacing=(10.0, 1.367188, 1.367188)):
    """
    Extracts a 3D surface mesh from the original DICOM volume using Marching Cubes.
    The segmentation mask is used as a threshold to isolate the left ventricle region.

    Parameters:
    - original_volume (ndarray): The 3D DICOM volume (slices, height, width).
    - segmented_volume (ndarray): The 3D binary segmentation mask (LV = 1, Background = 0).
    - spacing (tuple): (dz, dy, dx) voxel spacing for accurate scaling.

    Returns:
    - vertices (ndarray): Mesh vertices.
    - faces (ndarray): Mesh faces (triangles).
    """
    if original_volume.ndim != 3 or segmented_volume.ndim != 3:
        raise ValueError(f"Expected 3D input, but got shape {original_volume.shape}")

    thresholded_volume = np.where(segmented_volume > 0, original_volume, 0)

    valid_pixels = thresholded_volume[thresholded_volume > 0]
    if len(valid_pixels) == 0:
        print("Warning: No valid pixels in thresholded volume. Skipping.")
        return None, None

    level = np.percentile(valid_pixels, 50).item()

    vertices, faces, normals, values = marching_cubes(thresholded_volume, level=level, spacing=spacing)
    return vertices, faces

def compute_lv_metrics(vertices, faces, spacing):
    """
    Computes Left Ventricle volume and surface area from Marching Cubes output.
    
    Parameters:
    - vertices (ndarray): Extracted 3D points from marching cubes.
    - faces (ndarray): Triangular faces from marching cubes.
    - spacing (tuple): Voxel spacing (dz, dy, dx) in mm.

    Returns:
    - volume (float): LV volume in milliliters (mL).
    - surface_area (float): LV surface area in cm².
    """
    if vertices is None or faces is None:
        return None, None

    # Convert voxel spacing to cubic mm (scaling factor)
    voxel_volume = spacing[0] * spacing[1] * spacing[2]  # mm³

    # Compute LV Volume using Delaunay triangulation
    tri = Delaunay(vertices)
    volume = np.sum(np.abs(np.linalg.det(vertices[tri.simplices[:, :3]]))) / 6.0  # Volume in mm³
    volume *= 1e-3/100  # Convert mm³ to mL

    # Compute LV Surface Area (using skimage function)
    surface_area = mesh_surface_area(vertices, faces) * 1e-2  # Convert mm² to cm²

    return volume, surface_area

def plot_3d_mesh(vertices, faces, save_path=None):
    """
    Plots the extracted 3D mesh using Matplotlib.
    """
    if vertices is None or faces is None:
        return

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection="3d")

    mesh = Poly3DCollection(vertices[faces], alpha=0.6)
    mesh.set_edgecolor("k")
    ax.add_collection3d(mesh)

    # Set the viewing angle
    ax.view_init(elev=100, azim=180) 

    ax.set_xlim(vertices[:, 0].min(), vertices[:, 0].max())
    ax.set_ylim(vertices[:, 1].min(), vertices[:, 1].max())
    ax.set_zlim(vertices[:, 2].min(), vertices[:, 2].max())

    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_zlabel("Z-axis")
    ax.set_title("3D Mesh of Left Ventricle")

    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

def create_heartbeat_gif(original_volume, segmented_volume, save_gif_path="heartbeat.gif", frames_per_slice=20, spacing=(10.0, 1.367188, 1.367188)):
    """
    Generates a GIF of the beating heart by iterating through the cardiac cycle.
    Also computes LV volume and surface area for each frame.

    Parameters:
    - original_volume (ndarray): The 3D DICOM volume (frames, height, width).
    - segmented_volume (ndarray): The 3D segmentation mask (frames, height, width).
    - save_gif_path (str): Path to save the output GIF.
    - frames_per_slice (int): Number of frames per depth slice.
    - spacing (tuple): Voxel spacing.
    """
    frames = []
    temp_frame_paths = []
    lv_volumes = []
    lv_surface_areas = []

    num_frames = frames_per_slice  # Since we have 20 unique time frames
    slices_per_frame = original_volume.shape[0] // num_frames  # Number of spatial slices

    for t in range(num_frames):
        print(f"Processing frame {t + 1}/{num_frames}")

        # Select every 20th slice for the current time frame
        frame_original = original_volume[t::num_frames]  # Shape: (slices_per_frame, height, width)
        frame_segmented = segmented_volume[t::num_frames]  # Shape: (slices_per_frame, height, width)

        vertices, faces = extract_surface_marching_cubes(frame_original, frame_segmented, spacing)

        if vertices is None or faces is None:
            lv_volumes.append(None)
            lv_surface_areas.append(None)
            continue  # Skip frames with no valid segmentation

        # Compute LV volume and surface area
        volume, surface_area = compute_lv_metrics(vertices, faces, spacing)
        lv_volumes.append(volume)
        lv_surface_areas.append(surface_area)

        frame_path = f"frame_{t+1}.png"
        plot_3d_mesh(vertices, faces, save_path=frame_path)
        temp_frame_paths.append(frame_path)

    # Create GIF
    for frame_path in temp_frame_paths:
        frames.append(imageio.v2.imread(frame_path))
    imageio.mimsave(save_gif_path, frames, duration=0.1)
    print(f"Saved heartbeat animation as {save_gif_path}")

    # Print extracted LV metrics
    print("\nLV Volume (mL) over Time:")
    print(lv_volumes)

    print("\nLV Surface Area (cm²) over Time:")
    print(lv_surface_areas)

    plot_lv_metrics(lv_volumes, lv_surface_areas)

    return lv_volumes, lv_surface_areas

In [ ]:
normalized_volume = normalize_volume(volume)

# Crop the center 256x256 region
cropped_volume = crop_center(normalized_volume, crop_size=(120, 120))

# Apply center weighting
cropped_volume = apply_center_weighting(cropped_volume, alpha=3)

# Apply segmentation with border removal
segmented_volume = segment_heart_otsu(cropped_volume)

# Apply circular filtering to segmentation
filtered_segmentation = filter_circular_regions(segmented_volume, circularity_thresh=0.15, min_size=100)

# Apply watershed segmetation 
watershed_volume = improved_watershed(segmented_volume)

# Isolate the left ventricle
left_ventricle_isolation = LV_isolation(watershed_volume)

# Morphological opening and closing to remove small specks and fill holes
cleaned_volume = clean_segmentation(left_ventricle_isolation, radius=2)

# Output gif of cardiac left ventricle cycle from marching cubes reconstruction and output metrics for LV quantification
lv_volumes, lv_surface_areas = create_heartbeat_gif(cropped_volume, cleaned_volume)